### Initializing a Dictionary: 

First I initialized an empty dictionary named bikelane_lengths. This dictionary is intended to store the length of bike lanes within each neighborhood. The keys of this dictionary are neighborhood names (hood_name), obtained from data retrieved from the neighborhood datasets.

In [ ]:
# Load GeoJSON data for bikelanes
with open('data/2019-bike-lanes.geojson') as f:
    bikelanes_data = json.load(f)

# Initialize a dictionary to store lane lengths
bikelane_lengths = {hood_name: 0.0 for hood_name in hoods}

### Calculating Lengths
After the dictionary was created to store the length data, I created a loop to iterate through the bike lane features from our bike lane dataset. For each neighborhood I used the shapely.intersects() method to determine if the bike lane intersects (overlaps) the neighborhood. If it did, I used the shapely.intersection() method to get the LineString or MultiLineString object that stores the part of the bike lane that overlaps the neighborhood. The LineString objects has a length property which I then add to the existing length sum.

In [ ]:
# Iterate over each bike lane feature
for feature in bikelanes_data['features']:
    bikelane_geometry = shape(feature['geometry'])
    # Iterate through each neighborhood
    for hood_name, polygon in zip(hoods, polygons):
        # Check if the bike lane intersects with the neighborhood polygon
        if bikelane_geometry.intersects(polygon):
            # Calculate the length of the intersection
            intersection = bikelane_geometry.intersection(polygon)
            bikelane_lengths[hood_name] += intersection.length

The length property of LineString objects in Shapely measures the distance along the lines, just like measuring a string with a ruler. The scale of this length is based on the coordinate system used. However, this measurement is like a straight line drawn between points and doesn't consider the Earth's round shape for geographic data, therefore it is not useful for us to conceptualize.

### Compiling for Writing
After the calculations were made, I wrote code to add it into the dictionary holding all the data for it to be written to a json file.

In [ ]:
# Update merged_results with bike lane lengths
for entry in merged_results["neighborhoods"]:
    hood_name = entry["name"]
    entry["bikelane_length"] = bikelane_lengths[hood_name]

### Normalizing Values and Creating a Single Metric Score
I created code to min-max normalize each score which would give us values between 0 and 1 for each score. This will allow us to create a composite score comprized of the sum of all three normalized metrics.

In [ ]:
# Normalize the data
max_parks = max(neighborhood['parks'] for neighborhood in neighborhoods)
max_prt_stops = max(neighborhood['prt-stops'] for neighborhood in neighborhoods)
max_bikelane_length = max(neighborhood['bikelane_length'] for neighborhood in neighborhoods)

for neighborhood in neighborhoods:
    neighborhood['parks_normalized'] = neighborhood['parks'] / max_parks
    neighborhood['prt_stops_normalized'] = neighborhood['prt-stops'] / max_prt_stops
    neighborhood['bikelane_length_normalized'] = neighborhood['bikelane_length'] / max_bikelane_length


### Weighting the Scores
I also created code to weight each score to assign more or less importance on individual metrics. These were the values we settled on.

- Parks: 30%
- Bus Stops: 40%
- Bike Lane Length: 30%

In [ ]:
# Assign weights 
weights = {
    'parks': .3,
    'prt_stops': .4,
    'bikelane_length': 0.3
}

# Calculate composite score
for neighborhood in neighborhoods:
    neighborhood['composite_score'] = (
        weights['parks'] * neighborhood['parks_normalized'] +
        weights['prt_stops'] * neighborhood['prt_stops_normalized'] +
        weights['bikelane_length'] * neighborhood['bikelane_length_normalized']
    )

### Conclusion
The data-driven determination for best neighborhood being Squirrel Hill South is different from my personal best neighborhood pick. I would say Shadyside is the best neighborhood. I live in Shadyside currently and love that its close to many other neighborhoods, has many grocery stores and shops, and lots of trees. It also has a relatively large amount of bike lanes. I think that Pittsburgh's topography makes it prone to having many small neighborhoods as land features make up many of the borders between them. The result is that by living in one neighborhood, you can easily reap the benefits of the others because you are so close. All that being said, the top 2 neighborhoods based on our data was Squirrel Hill South and East Liberty, which are both neighborhoods with lots of amenities and have a large population. Although I don't think these are the best neighborhoods personally, they would be great to live in.